In [ ]:
%pip install sentencepiece==0.1.97 # same as silnlp
%pip install nltk==3.7 # same as silnlp
%pip install sacrebleu==2.3.1 # same as silnlp
%pip install rouge_score
%pip install editdistance

In [ ]:
import sacrebleu
from nltk.translate.gleu_score import sentence_gleu
from nltk.translate.nist_score import corpus_nist
import nltk
from rouge_score import rouge_scorer
import sacrebleu
import editdistance

In [ ]:
# NLTK data
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
#path = "C:/mySIL/preprocessed/"     # For Windows
path = "/Users/laura/llmResearch/preprocessed/"
language = "western_chawma"
dataset = "test"

In [ ]:
with open(path + "/" + "unsloth_seallm_allTrainingData_1epoch_" + language + "_" + dataset + "_generated.txt", "r", encoding="utf-8") as file:
    trg_predictions = file.readlines()
with open(path + language + "/" + dataset + ".trg.detok.txt", "r", encoding="utf-8") as file:
    trg = file.readlines()
with open(path + language + "/" + dataset + ".src.detok.txt", "r", encoding="utf-8") as file:
    src = file.readlines() 

In [ ]:
print(src[0])
print(trg[0])
print(trg_predictions[0])

In [ ]:
# automatically remove prompt from results
#trg_predictions = [i[i.find(":")+2:] for i in trg_predictions] #32

for i in range(len(trg_predictions)):
  source_sentence = src[i][:-1] #remove new line character from the end of the source sentence
  prediction = trg_predictions[i]
  print(source_sentence)
  print(prediction)
  
  if prediction[:len(source_sentence)]==source_sentence: #prediction repeats the source sentence
    trg_predictions[i] = prediction[len(source_sentence)+1:] #remove source sentence from prediction
    print(trg_predictions[i])
  break

In [ ]:
pair_sys = trg_predictions
pair_refs = [trg]

scores = {}

bleu_score = sacrebleu.corpus_bleu(pair_sys,pair_refs,lowercase=True,tokenize="13a")
scores["BLEU"] = bleu_score.score

chrf3_score = sacrebleu.corpus_chrf(pair_sys, pair_refs, char_order=6, beta=3, remove_whitespace=True)
scores["chrF3"] = chrf3_score.score

chrfp_score = sacrebleu.corpus_chrf(pair_sys, pair_refs, char_order=6, beta=3, word_order=1, remove_whitespace=True, eps_smoothing=True)
scores["chrF3+"] = chrfp_score.score

chrfpp_score = sacrebleu.corpus_chrf(pair_sys, pair_refs, char_order=6, beta=3, word_order=2, remove_whitespace=True, eps_smoothing=True)
scores["chrF3++"] = chrfpp_score.score

spbleu_score = sacrebleu.corpus_bleu(pair_sys, pair_refs, lowercase=True,tokenize="flores200",)
scores["spBLEU"] = spbleu_score.score

In [ ]:
len(trg_predictions)

In [ ]:
scores

In [ ]:

# METEOR score to evaluate trnaslation quality (synonyms, alignment, stemming, etc.)
def compute_meteor(translations, references):
    scores = [meteor_score([ref.split()], trans.split()) for trans, ref in zip(translations, references)]
    return sum(scores) / len(scores)

# ROUGE score is sensitive to missing words which is useful for identifying missing words from translations
def compute_rouge(translations, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, trans) for trans, ref in zip(translations, references)]
    averaged_scores = {key: sum(score[key].fmeasure for score in scores) / len(scores) for key in scores[0]}
    return averaged_scores

# GLEU score to evaluate sentence level quality
def compute_gleu(translations, references):
    scores = [sentence_gleu([ref.split()], trans.split()) for trans, ref in zip(translations, references)]
    return sum(scores) / len(scores)

# TER score to evalute missing words, segment alignment, etc.
def compute_ter(translations, references):
    ter_score = sacrebleu.corpus_ter(translations, references).score
    return ter_score

# Edit Distance to identify misspelled words, punctuation, missing/extra word checking
def compute_edit_distance(translations, references):
    distances = [editdistance.eval(trans, ref) for trans, ref in zip(translations, references)]
    return sum(distances) / len(distances)

# precision and recall to measure how well the model captures keywords
def compute_precision_recall(translations, references):
    precisions = []
    recalls = []
    for trans, ref in zip(translations, references):
        reference_tokens = set(ref.split())
        translation_tokens = set(trans.split())
        common_tokens = reference_tokens & translation_tokens
        precision = len(common_tokens) / len(translation_tokens)
        recall = len(common_tokens) / len(reference_tokens)
        precisions.append(precision)
        recalls.append(recall)
    return sum(precisions) / len(precisions), sum(recalls) / len(recalls)

# NIST score (weighting rare n-grams more heavily)
def compute_nist(translations, references):
    tokenized_translations = [trans.split() for trans in translations]
    tokenized_references = [[ref.split()] for ref in references]
    nist_score = corpus_nist(tokenized_references, tokenized_translations, n=5)  # Use n-gram size of 5
    return nist_score


# Compute scores  
#scores["METEOR"] = compute_meteor(pair_sys, pair_refs[0])
rouge_scores = compute_rouge(pair_sys, pair_refs[0])
for key, value in rouge_scores.items():
    scores[f"ROUGE-{key.upper()}"] = value
scores["GLEU"] = compute_gleu(pair_sys, pair_refs[0])
scores["TER"] = compute_ter(pair_sys, pair_refs[0])
scores["Edit Distance"] = compute_edit_distance(pair_sys, pair_refs[0])
precision, recall = compute_precision_recall(pair_sys, pair_refs[0])
scores["Precision"] = precision
scores["Recall"] = recall
scores["NIST"] = compute_nist(pair_sys, pair_refs[0])

# Print the computed scores
for score_name, score_value in scores.items():
    print(f"{score_name}: {score_value}")



In [ ]:
import json

path = "/Users/laura/llmResearch/scores/"

def save_to_jsonl(file_path, data, language):

    data_with_language_name = {"language": language}
    data_with_language_name.update(data)
    
    # Save scores to JSONL file and not overwritting existing rows
    with open(file_path, 'a') as file:
        file.write(json.dumps(data_with_language_name) + "\n")

data = scores
file_path = path + language + "_" + dataset + "_scores.jsonl"
save_to_jsonl(file_path, data, language)